In [659]:
import pandas as pd
import numpy as np
import math
import os

# import for time, and file change
import json
from PIL import Image

# frequent updating
import time
import random

In [660]:
# define filepaths
workspace = "/Users/advaysingh/Documents/projects/hindi_classification/" 
#print("Current workspace:", workspace)

data = os.path.join(workspace, 'data/Hindi/')
dict_lib = os.path.join(workspace, 'data/dict.csv')
img_path = os.path.join(workspace, 'server/snapshot.png')
out_file = os.path.join(workspace, 'server/outputs.json')


In [661]:
# create train/test dicts with files and labels

def create_dict(x: str) -> dict:
    x_dict = {}
    i = 0
    for dir in os.listdir(os.path.join(data, x)):
        for file in os.listdir(os.path.join(data, x, dir)):
            x_dict[os.path.join(data, x, dir, file)] = i
        i += 1
    return x_dict

# make pandas df

train_df = pd.DataFrame.from_dict(create_dict('Train'), orient='index')
test_df = pd.DataFrame.from_dict(create_dict('Test'), orient='index')

#print(train_df)
#df_temp = train_df[0].drop_duplicates()
# print(df_temp)

In [611]:
# Activations class
class Activation:
    # can add more types
    def __init__(self, act_type: str) -> None:
        self.act = act_type

    def print_act(self) -> None:
        return self.act
    
    def compute(self, z) -> list:

        # Sigmoid function
        if (self.act == 'sigmoid'):
            n = []
            for val in (z):
                n.append(1.0 / (1.0 + np.exp(val)))
            return n
        
        # ReLU function
        elif (self.act == 'relu'):
            vals = []
            for val in z:
                if val < 0.0:
                    vals.append(0.0)
                else:
                    vals.append(1.0)
            return vals

    def prime(self, z: list):

        # Sigmoid prime
        if (self.act == 'sigmoid'):
            sigs = self.compute(z)
            vals = []
            for val in sigs:
                vals.append(val * (1 - val))
            return vals
        
        # ReLU prime
        if (self.act == 'relu'):
            vals = []
            for val in z:
                if val == 0:
                    vals.append(0.0)
                else:
                    vals.append(1.0)
            return vals


def softmax(costs: list) -> list:
    exp_vals = []
    for cost in np.array(costs):
        exp_vals.append(np.exp(cost))
    return_vals = []
    for i in range(len(exp_vals)):
        return_vals.append(exp_vals[i] / sum(exp_vals))
    return return_vals

def softmax_prime(costs: list) -> list:
    softs = softmax(costs)
    vals = []
    for val in softs:
        vals.append(val * (1 - val))
    return vals

n = softmax([2, 3, 4])
print(n)

[0.09003057317038046, 0.24472847105479767, 0.6652409557748219]


In [618]:

"""class Model:
    def __init__(self, train_df: pd.DataFrame,
                  act: Activation, layers: list, step: float, stochastic: bool, load_perams: bool) -> None:
        self.train_df = train_df

        # create activation function
        self.act = act
        self.layers = layers

        # init weight and bias with # layers np arrays
        # Using the Xavier weight init method https://machinelearningmastery.com/weight-initialization-for-deep-learning-neural-networks/
        self.all_weights = [None] * (int(len(layers)))
        self.all_bias = [None] * (int(len(layers)))
        if (load_perams):
            layer = 0
            weightlist = os.listdir(os.path.join(workspace, 'data', 'Hyper_p/Weights'))
            for file in sorted(weightlist, key=lambda s: s.lower()):
                self.all_weights[layer] = pd.read_csv(os.path.join(workspace, 'data', 'Hyper_p/Weights', file)).to_numpy()
                #self.all_weights[layer] = np.delete(self.all_weights, [0], axis=1)
                print(self.all_weights[layer].shape)
                layer += 1
            layer = 0
            biaslist = os.listdir(os.path.join(workspace, 'data', 'Hyper_p/Biases'))
            for file in sorted(biaslist, key=lambda s: s.lower()):
                self.all_bias[layer] = pd.read_csv(os.path.join(workspace, 'data', 'Hyper_p/Biases', file)).to_numpy()
                layer += 1
            print(self.all_weights[0])
        else:
            for i in range(len(layers)):
                self.all_weights[i] = self.random_arrs(i)
                if i == 0:
                    self.all_bias[i] = np.random.uniform(low=-0.03125, high=0.03125, size=(self.layers[i],1))
                else:
                    bias_val = 1 / math.sqrt(self.layers[i - 1])
                    self.all_bias[i] = np.random.uniform(low=-bias_val, high=bias_val, size=(self.layers[i],1))
            #for val in self.all_weights:
               # print(val.shape)

        # init learn speed
        self.speed = step

        # determine gradient decent type
        self.stochastic = stochastic

    def random_arrs(self, layer: int):
        if (layer == 0):
            return(np.random.uniform(low=-0.03125, high=0.03125, size=(self.layers[layer],1024)))
        else:
            weight_val = 1 / math.sqrt(self.layers[layer - 1])
            return(np.random.uniform(low=-weight_val, high=weight_val, size=(self.layers[layer], self.layers[layer-1])))

    def img_to_np(self, dir) -> np.array:
        return np.array(Image.open(dir)).flatten()


    def print_weights(self) -> None:
        for arr in self.all_weights:
            print(arr.shape)

    def prop_forward(self, inputs: list, layer: int) -> list:
        a_vals = [0.0] * int(self.layers[layer])
        a_vals = np.dot(self.all_weights[layer], pd.DataFrame(inputs)) + self.all_bias[layer]
        if (layer == len(self.layers) - 1):
            return (softmax(a_vals))
        return(self.act.compute(a_vals))
    



    def prop_backword(self, x: list, label: list):
        delta_w = [None] * len(self.layers)
        delta_b = [None] * len(self.layers)
        inputs = []
        for i in range(len(self.layers)):
            inputs.append(x)
            x = self.prop_forward(x, i)
        error = [a - b for a, b in zip(x, label)]
        vals = np.array([a * b for a, b in zip(error, softmax_prime(x))])
        for i in range(len(self.layers), 0, -1):
            delta_w[i - 1] = np.dot(pd.DataFrame(vals), pd.DataFrame(inputs[i - 1]).transpose())
            delta_b[i - 1] = vals
            vals = np.dot(pd.DataFrame(self.all_weights[i - 1]).transpose(), pd.DataFrame(vals))
            vals = [a * b for a, b in zip(vals, self.act.prime(inputs[i - 1]))]
            #print(np.dot(pd.DataFrame(vals), pd.DataFrame(inputs[0]).transpose()))
        #print(delta_w[0])
        return delta_w, delta_b

            


    # potentially reconsider for time complexity
    def vals_for_softmax(self, index: int) -> list:
        vals = self.train_df[0].unique()
        for i in range(len(vals)):
            if vals[i] != index:
                vals[i] = 0
            else:
                vals[i] = 1
        return vals


    def train(self, epochs: int) -> None:
        length_set = len(self.train_df)
        for epoch in  range(epochs):
            delta_w = [np.zeros_like(array) for array in self.all_weights]
            delta_b = [np.zeros_like(array) for array in self.all_bias]
            count = 0
            for img_dir, row in self.train_df.iterrows():
                print("img", count)
                count += 1
                temp_w, temp_b = self.prop_backword(self.img_to_np(img_dir).tolist(), self.vals_for_softmax(row[0]))
                for i in range(len(self.layers)):
                    delta_b[i] += temp_b[i]
                    delta_w[i] += temp_w[i]
            delta_w = [array / length_set for array in delta_w]
            delta_b = [np.array(array) / length_set for array in delta_b]
            for i in range(len(self.layers)):
                self.all_weights[i] += delta_w[i]
                self.all_bias[i] += delta_b[i]
            print("validating")
            print("epoch: ", epoch)
            self.validate(self.train_df)

    def pred(self, img: np.array) -> int:
        for i in range(len(self.layers)):
            img = self.prop_forward(img, i)
        return img

    def validate(self, test_df: pd.DataFrame) -> None:
        count, countright = 0, 0
        for img_dir, row in test_df.iterrows():
            outputs = self.pred(self.img_to_np(img_dir))
            actual = list(self.vals_for_softmax(row[0]))
            if (outputs.index(max(outputs))) == (actual.index(max(actual))):
                countright += 1
            count += 1
            accuracy = (float(countright) / float(count)) * 100
        print("count: ", count, "countright: ", countright, ". Accuracy: ", accuracy, "%")

    def save(self) -> None:
        hp_data = '/Users/advaysingh/Documents/projects/hindi_classification/data/hyper_p'
        hp_data_biases, hp_data_weights = hp_data + '/Biases', hp_data + '/Weights'
        os.makedirs(hp_data)
        os.makedirs(hp_data_biases)
        os.makedirs(hp_data_weights)
        for layer in range(len(self.layers)):
            df_weights = pd.DataFrame(self.all_weights[layer])
            df_bias = pd.DataFrame(self.all_bias[layer])
            weights_file_name = "layer_" + str(layer) + "weights.csv"
            bias_file_name = "layer_" + str(layer) + "bias.csv"
            weights_file = open(os.path.join(hp_data_weights, weights_file_name), "x")
            bias_file = open(os.path.join(hp_data_biases, bias_file_name), "x")
            df_weights.to_csv(weights_file, index=False)
            df_bias.to_csv(bias_file, index=False)


    def print_weights(self, layer: int) -> None:
        for row in self.all_weights[layer]:
            for weight in row:
                print(weight)"""



In [931]:
def activate(z: np.array) -> np.array:
    """Compute the sigmoid activation function."""
    return 1 / (1 + np.exp(-z))

def act_prime(z: np.array) -> np.array:
    """Compute the derivative of the sigmoid activation function."""
    ap = activate(z)
    return ap * (1 - ap)

def softmax(z: np.array) -> np.array:
    e_z = np.exp(z - np.max(z)) # Improved stability
    return e_z / e_z.sum(axis=0)

def softmax_prime(z: np.array) -> np.array:
    softmax_vals = softmax(z)
    # For each i, j in softmax output vector size: [si * (1 - si)] if i == j, otherwise: -si * sj
    return np.diag(softmax_vals) - np.outer(softmax_vals, softmax_vals)

class Model:
    def __init__(self, layers: list, load_perams: bool) -> None:
        self.train_df = train_df
        self.layers = layers

        self.all_weights = [None] * len(self.layers)
        self.all_bias = [None] * len(self.layers)

        # read in weights if load perams
        if (load_perams):
            layer = 0
            weightlist = os.listdir(os.path.join(workspace, 'data', 'Hyper_p/Weights'))
            for file in sorted(weightlist, key=lambda s: s.lower()):
                self.all_weights[layer] = (pd.read_csv(os.path.join(workspace, 'data', 'Hyper_p/Weights', file)).to_numpy())
                layer += 1
            layer = 0
            biaslist = os.listdir(os.path.join(workspace, 'data', 'Hyper_p/Biases'))
            for file in sorted(biaslist, key=lambda s: s.lower()):
                self.all_bias[layer] = (pd.read_csv(os.path.join(workspace, 'data', 'Hyper_p/Biases', file)).to_numpy())
                layer += 1

        else:
            for i in range(len(layers)):
                self.all_weights[i] = self.random_arrs(i)
                if i == 0:
                    self.all_bias[i] = np.random.uniform(low=-0.03125, high=0.03125, size=(self.layers[i],1))
                else:
                    bias_val = 0.5 # TODO reconsider
                    self.all_bias[i] = np.random.uniform(low=-bias_val, high=bias_val, size=(self.layers[i],1))
    

    def random_arrs(self, layer: int):
        if (layer == 0):
            return(np.random.uniform(low=-0.03125, high=0.03125, size=(self.layers[layer],1024)))
        else:
            weight_val = 1 / math.sqrt(self.layers[layer - 1])
            return(np.random.uniform(low=-weight_val, high=weight_val, size=(self.layers[layer], self.layers[layer-1])))

    def img_to_np(self, dir) -> np.array:
        return np.array(Image.open(dir)).flatten()
    
    def prop_forward(self, x: np.ndarray):
        activations = [x] #activations.size > zs
        zs = []
        for i in range(len(self.layers)):
            x = np.matmul(self.all_weights[i], x) + self.all_bias[i]
            zs.append(x)
            if (i == len(self.layers) - 1):
                x = softmax(x)
            else:
                x = activate(x)
            activations.append(x)
        return activations, zs


    
    """def prop_backward(self, x: np.ndarray, y: np.ndarray):
        delta_b = [np.zeros(b.shape) for b in self.all_bias]
        delta_w = [np.zeros(w.shape) for w in self.all_weights]
        activations, zs = self.prop_forward(x)
        delta = activations[-1] - y #* softmax_prime(zs[-1]) #use zs delta
        delta_b[-1] = delta
        delta_w[-1] = np.dot(delta, activations[-2].transpose())
        # loop through layers and now apply the sigmoid prime
        for i in range(2, len(self.layers)):
            sp = act_prime(zs[-i]) # No need to print here
            delta = np.dot(self.all_weights[-i + 1].transpose(), delta) * sp 
            delta_b[-i] = delta
            delta_w[-i] = np.dot(delta, activations[-i - 1].T)
        return delta_w, delta_b"""
    def prop_backward(self, x_batch: np.ndarray, y_batch: np.ndarray):
        # Initialize gradients as zero
        delta_b_sum = [np.zeros(b.shape) for b in self.all_bias]
        delta_w_sum = [np.zeros(w.shape) for w in self.all_weights]

        # Loop over each example in the batch
        for x, y in zip(x_batch, y_batch):
            activations, zs = self.prop_forward(x)
            delta = activations[-1] - y.reshape(46,1)  # Assuming cost_derivative is defined elsewhere

            # Gradients for output layer
            delta_b_sum[-1] += delta
            delta_w_sum[-1] += np.dot(delta, activations[-2].transpose())

            # Gradients for hidden layers
            for i in range(2, len(self.layers)):
                z = zs[-i]
                sp = act_prime(z)
                delta = np.dot(self.all_weights[-i + 1].transpose(), delta) * sp
                delta_b_sum[-i] += delta
                delta_w_sum[-i] += np.dot(delta, activations[-i - 1].T)

        # Average the gradients over the mini-batch
        num_examples = len(x_batch)
        delta_b_avg = [db_sum / num_examples for db_sum in delta_b_sum]
        delta_w_avg = [dw_sum / num_examples for dw_sum in delta_w_sum]

        # Return the average gradients
        return delta_w_avg, delta_b_avg
    
    def update_mini_batch(self, mini_batch, learning_rate):
        # aggregate gradients from the mini_batch
        delta_w, delta_b = self.prop_backward(mini_batch[0], mini_batch[1])
    
        # update weights and biases
        self.all_weights = [w - (learning_rate * dw) for w, dw in zip(self.all_weights, delta_w)]
        self.all_bias = [b - (learning_rate * db) for b, db in zip(self.all_bias, delta_b)]
    

    def train(self, X_train, y_train, epochs, learning_rate, mini_batch_size):
        n = len(X_train)

        # Convert to NumPy arrays if necessary
        X_train_np = np.array(X_train) if not isinstance(X_train, np.ndarray) else X_train
        y_train_np = np.array(y_train) if not isinstance(y_train, np.ndarray) else y_train
    
        for epoch in range(epochs):
            # Shuffle the training data for each epoch
            permutation = np.random.permutation(n)
            X_train_shuffled = X_train_np[permutation]
            y_train_shuffled = y_train_np[permutation]
        
            # Partition training data into mini-batches
            mini_batches = [
                (X_train_shuffled[k:k+mini_batch_size], y_train_shuffled[k:k+mini_batch_size])
                for k in range(0, n, mini_batch_size)
            ]
        
            # Update the model's weights with each mini-batch
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, learning_rate)

            new_loss = self.calc_squared_loss(X_train, y_train)
            print(f"Epoch {epoch}, mean squared loss: {new_loss}")
    
    
    """def train(self, X_train, y_train, epochs=100, lr=1, verbose=True) -> None:
            # Shuffle the data at the start of each epoch (optional)
        e = 0
        while (e < epochs):
            rng = np.random.permutation(1024)
            d_x = 1024
            d_y = 46
            for i in range(100):
                x = X_train[rng[i]].reshape((d_x, 1))
                y = y_train[rng[i]].reshape((d_y, 1))
                delta_w, delta_b = self.prop_backward(x, y)
                for i in range(len(self.all_weights)):
                    self.all_weights[i] -= lr * delta_w[i]
                    self.all_bias[i] -= lr * delta_b[i]

            new_loss = self.calc_squared_loss(X_train, y_train)
            if verbose:
                print(f"Epoch {e}, mean squared loss: {new_loss}")
            e+=1"""

    def calc_squared_loss(self, X, y):
        squared_errors = []
        for i in range(len(X)):
            y_pred, _ = self.prop_forward(X[i].reshape(1024, 1))  # prop_forward should give the prediction
            squared_error = np.sum((y[i].reshape(46, 1) - y_pred[-1]) ** 2) / 2 
            squared_errors.append(squared_error)
        
        # Calculate mean squared error over all examples.
        mse = np.mean(squared_errors)
        return mse

    def pred(self, img: np.array):
        img, _ = self.prop_forward(img)
        return list(img[-1])
    
    def vals_for_softmax(self, index: int) -> np.array:
        """ Helper function to create softmax values"""
        vals = np.zeros(46)
        vals[index] = 1
        vals.reshape(46, 1)
        return vals
    
    def validate(self, test_df: pd.DataFrame) -> None:
        count, countright = 0, 0
        for img_dir, row in test_df.iterrows():
            img = np.array(Image.open(img_dir).convert("L").resize((32, 32))).flatten().reshape(1024, 1)
            outputs = list(self.pred(img))
            actual = list(self.vals_for_softmax(row[0]))
            if (list(self.pred(outputs)).index(list(max(self.pred(outputs)))) - list(actual).index(list(max(actual))) < 2):
                countright += 1
            count += 1
            accuracy = (float(countright) / float(count)) * 100
        print("count: ", count, "countright: ", countright, ". Accuracy: ", accuracy, "%")

    def save(self) -> None:
        hp_data = '/Users/advaysingh/Documents/projects/hindi_classification/data/hyper_p'
        hp_data_biases, hp_data_weights = hp_data + '/Biases', hp_data + '/Weights'
        os.makedirs(hp_data)
        os.makedirs(hp_data_biases)
        os.makedirs(hp_data_weights)
        for layer in range(len(self.layers)):
            df_weights = pd.DataFrame(self.all_weights[layer])
            df_bias = pd.DataFrame(self.all_bias[layer])
            weights_file_name = "layer_" + str(layer) + "weights.csv"
            bias_file_name = "layer_" + str(layer) + "bias.csv"
            weights_file = open(os.path.join(hp_data_weights, weights_file_name), "x")
            bias_file = open(os.path.join(hp_data_biases, bias_file_name), "x")
            df_weights.to_csv(weights_file, index=False)
            df_bias.to_csv(bias_file, index=False)
    

In [932]:
def vals_for_softmax(index: int) -> np.array:
        """ Helper function to create softmax values"""
        vals = np.zeros(46)
        vals[index] = 1
        vals.reshape(46, 1)
        return vals
test_df_sample = test_df.sample(500)


X_train, y_train = [], []
for img_dir, row in train_df.iterrows():
            X_train.append(np.array(Image.open(img_dir)).flatten().reshape(1024, 1))
            y_train.append(vals_for_softmax(row[0]))

In [937]:

cnn = Model([300, 100, 46], False)

cnn.train(X_train, y_train, 5000, 0.5, mini_batch_size=int(300))


Epoch 0, mean squared loss: 0.4202735903651491
Epoch 1, mean squared loss: 0.3311629616681811
Epoch 2, mean squared loss: 0.2939424643262792
Epoch 3, mean squared loss: 0.27262557224676176
Epoch 4, mean squared loss: 0.2578985695381331
Epoch 5, mean squared loss: 0.24976406278643373
Epoch 6, mean squared loss: 0.24045950648587333
Epoch 7, mean squared loss: 0.23281125685917167
Epoch 8, mean squared loss: 0.22752016746863832
Epoch 9, mean squared loss: 0.22018771887618302
Epoch 10, mean squared loss: 0.2148399322830054
Epoch 11, mean squared loss: 0.2091867436543063
Epoch 12, mean squared loss: 0.20674622459243705
Epoch 13, mean squared loss: 0.2008296856864922
Epoch 14, mean squared loss: 0.19567279901059612
Epoch 15, mean squared loss: 0.19092184426832273
Epoch 16, mean squared loss: 0.18830549743266348
Epoch 17, mean squared loss: 0.18356197468846489
Epoch 18, mean squared loss: 0.17859175265429153
Epoch 19, mean squared loss: 0.17714996533554667
Epoch 20, mean squared loss: 0.173185

KeyboardInterrupt: 

In [938]:
cnn.save()

In [929]:
cnn = Model([10, 46], True)
cnn.validate(test_df)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 46 is different from 1024)

In [434]:
cnn.save()

In [941]:
import json
import time
import random

from PIL import Image

cnn = Model([100, 50, 46], True)
"""outs = cnn.pred(cnn.img_to_np(img_path))
index = outs.index(max(outs))
new_english_value = pd.read_csv(os.path.join(workspace, 'data', 'dict.csv'))[index][1]"""

def update_json_file(hindi_value, english_value):
    with open(out_file, 'r') as file:
        data = json.load(file)
    data['hindi'] = hindi_value
    data['english'] = english_value
    with open(out_file, 'w') as file:
        json.dump(data, file, indent=4)

# Define a list of possible values if applicable
hindi_values = ["हिं", "नया", "मूल्य", "उदाहरण"]
english_values = ["Eng", "New", "Value", "Example"]

# Loop to update the JSON file every second with a random value
try:
    while True:
        # Generate a random value from the lists
        new_hindi_value = random.choice(hindi_values)
        #new_english_value = random.choice(english_values)
        img = np.array(Image.open(img_path).convert("L").resize((32, 32))).flatten().reshape(1024, 1)
        index = list(cnn.pred(img)).index(list(max(cnn.pred(img)))) #.index(max(cnn.pred(img)))
        new_english_value = str(pd.read_csv(os.path.join(workspace, 'data', 'dict.csv')).iloc[index -1, 1])

        update_json_file(new_hindi_value, new_english_value)
        
        # Print the new values (for debug purposes)
        # print(f'Updated JSON file with hindi: {new_hindi_value}, english: {new_english_value}')

        # read in and resize img
 
        # read in image for debugging
        """img.show("img to classify")
        print(img.format)
        print(img.mode)
        print(img.size)"""

        # create np array and resize
        """img_data = np.array(img).flatten()
        print(img_data.shape)
        print(img_data)"""

        
        time.sleep(1)  # Wait for 1 second
except KeyboardInterrupt:
    print("Update stopped by user.")

Update stopped by user.
